<a href="https://colab.research.google.com/github/kabatyy/deepCaptioning/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>